## Learning problem

This file shows an example of a robust learning problem where the uncertainty set is learned.

In [1]:
import numpy as np
import numpy.random as npr
import numpy.testing as npt
import scipy as sc
import torch
import cvxpy as cp

# from tests.settings import SOLVER
from sklearn.model_selection import train_test_split

from lropt.parameter import Parameter
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter
from lropt.uncertainty_sets.ellipsoidal import Ellipsoidal
#from tests.settings import TESTS_ATOL as ATOL
#from tests.settings import TESTS_RTOL as RTOL

In [2]:
def gen_demand_intro(N, seed):
    """
    This function generates random demand.
    """
    np.random.seed(seed)
    sig = np.array([[0.3, -0.4], [-0.5, 0.1]])
    mu = np.array((0.3, 0.3))
    d_train = np.random.multivariate_normal(mu, sig, N)
    # d_train = np.exp(d_train)
    return d_train

In [3]:
n = 2
N = 100
norms = npr.multivariate_normal(np.zeros(n), np.eye(n), N)
data = np.exp(norms)
kappa = -0.01
seed = 15
np.random.seed(seed)
dist = (np.array([25, 10, 60, 50, 40, 30, 30, 20,
        20, 15, 10, 10, 10, 10, 10, 10])/10)[:n]
y_data = np.random.dirichlet(dist, 10)
y = Parameter(n, data=y_data)

data = gen_demand_intro(600, seed=15)
u = UncertainParameter(n,uncertainty_set=Ellipsoidal(p=2,data=data))
# Formulate the Robust Problem
x = cp.Variable(n)
t = cp.Variable()
        

objective = cp.Minimize(t + 0.2*cp.norm(x - y, 1))
constraints = [-x@u <= t, cp.sum(x) == 1, x >= 0]
eval_exp = -x @ u + 0.2*cp.norm(x-y, 1)
prob = RobustProblem(objective, constraints, eval_exp=eval_exp)
test_p = 0.1
s = 5
train, _ = train_test_split(data, test_size=int(
    data.shape[0]*test_p), random_state=s)
init = sc.linalg.sqrtm(sc.linalg.inv(np.cov(train.T)))
-init@np.mean(train, axis=0)
np.random.seed(15)
initn = np.random.rand(n, n) + 0.1*init + 0.5*np.eye(n)
init_bvaln = -initn@(np.mean(train, axis=0) - 0.3*np.ones(n))

result = prob.train(lr=0.01, num_iter=100, momentum=0.8,
                            optimizer="SGD",
                            seed=s, init_A=initn, init_b=init_bvaln,
                            init_lam=0.5, init_mu=0.01,
                            mu_multiplier=1.001, init_alpha=0., test_percentage=test_p, kappa=kappa,
                            n_jobs=8, random_init=True, num_random_init=2, position=True)

/var/folders/p_/cgtqpk9j7kq74n_ft0y8yvxr0000gp/T/ipykernel_34551/2031112850.py:8: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  d_train = np.random.multivariate_normal(mu, sig, N)

run 0: test value -0.104, violations 0.0: 100%|██████████| 100/100 [00:03<00:00, 30.78it/s]


In [5]:
print(f"Test value = {result.df_test['Test_val'].to_string()}")
print(f"Probability violations test = {result.df_test['Probability_violations_test'].to_string()}")

Test value = 0   -0.204291
1   -0.199478
2   -0.183228
3   -0.170275
4   -0.158126
5   -0.142884
6   -0.131874
7   -0.125228
8   -0.119399
9   -0.116393
Probability violations test = 0    0.116667
1    0.066667
2        0.05
3    0.016667
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0


In [ ]:
print(results.df_test["Test_val"])